# LUAD CPTAC analysis

In [1]:
import pandas as pd
import numpy as np
from msresist.pre_processing import preprocessing
from msresist.clustering import MassSpecClustering
import matplotlib.pyplot as plt

In [2]:
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 1000)

## 1 - Import and Preprocess Data

In [3]:
# X = preprocessing(CPTAC=True, log2T=True)
X = pd.read_csv("msresist/data/MS/CPTAC/CPTAC-preprocessedMotfis.csv")

d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

## 2 - Run Model

Test speed of generating the motif pam250 scores:

In [4]:
%%time
from msresist.sequence_analysis import MotifPam250Scores
n = 5000
seqs = [s.upper() for s in X["Sequence"][:n]]
print(len(seqs))
MotifPam250Scores(seqs)

5000
CPU times: user 1min 13s, sys: 2.47 s, total: 1min 16s
Wall time: 1min 10s


array([[ 31,  13,   7, ...,  -1,  -1, -12],
       [ 13,  27,  13, ...,  -1,  -1, -17],
       [  7,  13,  27, ...,  -6,  -6, -13],
       ...,
       [ -1,  -1,  -6, ...,  37,  37,  17],
       [ -1,  -1,  -6, ...,  37,  37,  17],
       [-12, -17, -13, ...,  17,  17,  54]])

Run model:

In [5]:
# Reduce data set?
npept = 5000
d = d.iloc[:, :npept]
i = i.iloc[:npept, :]

#Drop NaN?
# d = d.iloc[:8, :].dropna(axis=1)
# i = i.iloc[d.columns, :]

In [6]:
%%time
distance_method = "PAM250"
ncl = 5
SeqWeight = 100

MSC = MassSpecClustering(i, ncl, GMMweight=SeqWeight, distance_method=distance_method, n_runs=1).fit(d, "NA")

start initialization...
gmm initialized
N_ITER:  0
-126.92267744249473
N_ITER:  1
-64.84839037593187
N_ITER:  2
12.531911548075971
N_ITER:  3
38.85234090235834
N_ITER:  4
50.24342530493183
N_ITER:  5
60.34873556674366
N_ITER:  6
66.2452387580936
N_ITER:  7
76.09472865279862
N_ITER:  8
91.60479154480186
N_ITER:  9
111.04523091445672
N_ITER:  10
122.41163024450063
N_ITER:  11
125.03124595378945
N_ITER:  12
126.63754123204569
N_ITER:  13
128.45769232819228
N_ITER:  14
128.7077133960794
N_ITER:  15
128.3939593070196
N_ITER:  16
128.54438605866437
N_ITER:  17
128.56774307146432
N_ITER:  18
128.66431565425052
N_ITER:  19
128.80301886024978
N_ITER:  20
128.62812468119353
N_ITER:  21
128.83356605431956
N_ITER:  22
128.82887377863793
N_ITER:  23
128.82887377863793
CPU times: user 6min 51s, sys: 4.33 s, total: 6min 56s
Wall time: 6min 42s


In [7]:
centers = MSC.transform(d)